# XGBoost Baseline

## Package Importing

In [19]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import SplineTransformer, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np
import xgboost as xgb

## Data Importing

In [53]:
# ——— load ———
df_train = pd.read_csv(
    filepath_or_buffer='../data/raw/train.csv', 
    index_col='id',
)
df_test  = pd.read_csv('../data/raw/test.csv', index_col='id')

# Preprocessing

# Parsing out episode title.

In [11]:
df_train['Episode_Number'] = (
    df_train
        ['Episode_Title']
        .str.split(' ') # split based on space so that each element is a list ['Episode','12']
        .apply(lambda lst: lst[1])
        .astype(int)
)

In [12]:
df_train = df_train.drop(columns='Episode_Title')

# Cyclic encoding of day

In [ ]:
{
    'Monday':0.1,
    'Tuesday':0.2,
    'Wednesday':0.3,
    'Thusday':,
    'Friday':0.4,
    'Saturday':0.5,
    'Sunday':0.6

}

,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Number
id,,,,,,,,,,,
0,Mystery Matters,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998,98
1,Joke Junction,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241,26
2,Study Sessions,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531,16
3,Digital Digest,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824,45
4,Mind & Body,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031,86
...,...,...,...,...,...,...,...,...,...,...,...
749995,Learning Lab,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058,25
749996,Business Briefs,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242,21
749997,Lifestyle Lounge,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000,51


# Number of Ads

In [ ]:
mean_number_of_ads = df_train['Number_of_Ads'].mean()
df_train['Number_of_Ads'] = df_train['Number_of_Ads'].fillna(mean_number_of_ads)
df_train['Number_of_Ads'] = df_train['Number_of_Ads'].astype(int)

In [ ]:
df_train.columns

Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes'],
      dtype='object')

# Training and Test Set Creation

In [46]:
df_train.select_dtypes(include='number')

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,Episode_Number
id,,,,,,
0,NaN,74.81,NaN,0.0,31.41998,98
1,119.80,66.95,75.95,2.0,88.01241,26
2,73.90,69.97,8.97,0.0,44.92531,16
3,67.17,57.22,78.70,2.0,46.27824,45
4,110.51,80.07,58.68,3.0,75.61031,86
...,...,...,...,...,...,...
749995,75.66,69.36,NaN,0.0,56.87058,25
749996,75.75,35.21,NaN,2.0,45.46242,21
749997,30.98,78.58,84.89,0.0,15.26000,51


In [28]:
X_train = df_train.drop(columns='Listening_Time_minutes').to_numpy()
y_train = df_train['Listening_Time_minutes'].to_numpy()

# Model Training and Cross Validation


In [29]:
SEED = 42
NFOLDS = 5
np.random.seed

<cyfunction seed at 0x1128e5110>

In [33]:
xgb_model = xgb.XGBRegressor(
    objective='rmse',
    random_state=42,
    enable_categorical=True
)

In [34]:
scores = cross_val_score(
    xgb_model, 
    X=X_train, 
    y=y_train, 
    scoring="neg_mean_squared_error", 
    cv=5
)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/cbutler/repos/personal_repos/Kaggle_S5E4/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cbutler/repos/personal_repos/Kaggle_S5E4/.venv/lib/python3.9/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/Users/cbutler/repos/personal_repos/Kaggle_S5E4/.venv/lib/python3.9/site-packages/xgboost/sklearn.py", line 1143, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/Users/cbutler/repos/personal_repos/Kaggle_S5E4/.venv/lib/python3.9/site-packages/xgboost/skle